In [2]:
from ipycanvas import Canvas, hold_canvas
from Klasser.celle import *

# visual size of tiles
tile_size = 15
visual_space = 2
rect_size = tile_size - visual_space

# amount of tiles in each direction
tiles_hori = 10
tiles_vert = 7

# Canvas size
width = tile_size * tiles_hori
height = tile_size * tiles_vert

canvas = Canvas(height = height, width = width)
display(canvas)

def draw_world():
    # Draw grid
    canvas.fill_style = "gray"
    for x in range(tiles_hori):
        for y in range(tiles_vert):
            canvas.fill_rect(x * tile_size, y * tile_size, rect_size, rect_size)


with hold_canvas():
    draw_world()


Canvas(height=105, width=150)